In [1]:
import pandas as pd
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [23]:
def clean_vendor_report(file_path):
    # 1. Load and promote headers
    df = pd.read_excel(file_path, skiprows=9)
    df.columns = [str(c).strip().lower().replace(' ', '_') for c in df.iloc[0]]
    df = df[1:].reset_index(drop=True)

    # 2. Capture the Performance Name (Row 0, first column)
    performance_name = df.iloc[0, 0]

    # 3. Create a clean 'perf_date' column
    # We look at the 'capacity' column where the actual data row is (Row 1)
    perf_date = df['capacity'].iloc[1]

    # 4. FILTERING: This is the magic part
    # Keep ONLY rows where 'sales' is actually a number
    # This automatically kills the title rows, empty rows, and spacers
    df = df[pd.to_numeric(df['sales'], errors='coerce').notna()].copy()

    # 5. REMOVE TOTALS: In case 'Total' appeared in the numeric filter
    # We check the 'capacity' column (which now holds the 'Total' label)
    df = df[~df['capacity'].astype(str).str.contains("Total", case=False, na=False)]

    # 6. Add our captured info as proper columns
    df.insert(0, 'performance_name', performance_name)
    df.insert(1, 'perf_date', perf_date)

    # 7. Final Cleanup
    # Drop the old 'capacity' column if it's just a duplicate of date now, 
    # or keep it if it's relevant.
    df = df.dropna(axis=1, how='all')
    
    return df.reset_index(drop=True)

In [24]:
df = clean_vendor_report("/Users/harveys/Desktop/repos/dsr_modeling/MSO Charlie Li plays Brahms  Sales report_112648_1_20260128.xls")
df.head()

,perf_date,capacity,sales,reservations,available,performance_time,venue,suspend,event,paid,house,ccard,comp,net,inside,merchant_fee,gross,paid,comp,gross
0,MSO: Christian Li plays Brahms (2026),2026-10-09 19:30:00,800,0,800,532,0,4,0,31679,2132,853,34664,3,0,105,0,79,0,182
